## Training
A template Jupyter notebook to further train models.
Data is given in compressed form and extracted for training.

In [8]:
import os
import errno

import numpy as np

import deepcell

In [9]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

norm_method = 'std'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Sample mode settings
batch_size = 64  # number of images per batch (should be 2 ^ n)
win = (receptive_field - 1) // 2  # sample window size
balance_classes = True  # sample each class equally
max_class_samples = 1e6  # max number of samples per class

# Transformation settings
transform = 'watershed'
distance_bins = 4  # number of distance "classes"
erosion_width = 1  # erode edges

In [10]:
# create folder for this set of experiments
experiment_folder = "20190621_postprocessing"
MODEL_DIR = os.path.join("/data/models", experiment_folder)
LOG_DIR = '/data/logs'
if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [11]:
base_name = 'Point1_12_18_20190606_output_3_class_w_interior_and_watershed'
npz_name = '/data/npz_data/' + base_name + ".npz"

sample_model_name = base_name + "_watershed"

In [14]:
# Load the training data from NPZ into a numpy array
CHANNEL_AXIS = 3
training_data = np.load(npz_name)

X, y = training_data["X"], training_data["y"]
print("X.shape: {} & y.shape: {}".format(X.shape, y.shape))

X.shape: (3, 1024, 1024, 4) & y.shape: (3, 1024, 1024, 1)


In [15]:
from deepcell import model_zoo
from deepcell.training import train_model_sample

watershed_model = model_zoo.bn_feature_net_2D(
    receptive_field=receptive_field,
    n_channels=X.shape[CHANNEL_AXIS],
    n_features=distance_bins,
    n_conv_filters=64,
    n_dense_filters=400)

In [16]:
print(sample_model_name)

Point1_12_18_20190606_output_3_class_w_interior_and_watershed_watershed


In [ ]:
watershed_model = train_model_sample(
    model=watershed_model,
    dataset=npz_name, 
    model_name=sample_model_name,
    test_size=0.1,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=40,
    window_size=(win, win),
    transform=transform,
    distance_bins=distance_bins,
    erosion_width=erosion_width,
    balance_classes=True,
    max_class_samples=180000,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2),
    val_monitor=False,
    save_period=10)

using all data as training data
Using class weights of {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0}
X_train shape: (3, 1024, 1024, 4)
y_train shape: (3, 1024, 1024, 1)
Output Shape: (None, 4)
Number of Classes: 4
Training on 1 GPUs
the max_class_samples per image is 60000
analyzing image 0
the least represented class has 30524 examples
analyzing class 0
downsampling from 739622 examples per class
analyzing class 1
downsampling from 95868 examples per class
analyzing class 2
downsampling from 61355 examples per class
analyzing class 3
downsampling from 30524 examples per class
analyzing image 1
the least represented class has 37791 examples
analyzing class 0
downsampling from 687593 examples per class
analyzing class 1
downsampling from 121406 examples per class
analyzing class 2
downsampling from 80579 examples per class
analyzing class 3
downsampling from 37791 examples per class
analyzing image 2
the least represented class has 44239 examples
analyzing class 0
downsampling from 646714 examples p